This file is part of GaPSE
Copyright (C) 2022 Matteo Foglieni

GaPSE is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

GaPSE is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU
General Public License for more details.

You should have received a copy of the GNU General Public License
along with GaPSE. If not, see <http://www.gnu.org/licenses/>.

In [1]:
PATH_TO_GAPSE = "../";

In [2]:
include(PATH_TO_GAPSE * "src/GaPSE.jl");

In [3]:
using DelimitedFiles, Plots, Dierckx, SpecialFunctions
using Printf, LaTeXStrings

In [4]:
FILE_F_MAP = PATH_TO_GAPSE * "test/datatest/F_REFERENCE.txt"
FILE_PS = PATH_TO_GAPSE * "test/datatest/file_pk.txt"
FILE_BACKGROUND = PATH_TO_GAPSE * "test/datatest/WideA_ZA_background.dat"

z_min, z_max, θ_max = 0.05, 0.20, π / 2.0
params = GaPSE.CosmoParams(z_min, z_max, θ_max;
    Ω_b = 0.0489, Ω_cdm = 0.251020, h_0 = 0.70, s_lim=1e-2,
    IPS_opts = Dict(
        :fit_left_min => 1e-6, :fit_left_max => 3e-6,
        :fit_right_min => 1e1, :fit_right_max => 2e1),
    IPSTools_opts = Dict(
        :N => 1024, :fit_min => 0.05, :fit_max => 0.5, 
        :con => true, :k_min => 1e-8, :k_max => 10.0),
)

cosmo = GaPSE.Cosmology(params, FILE_BACKGROUND, FILE_PS, FILE_F_MAP);

GaPSE.parameters_used(stdout, cosmo)

###############
#    GaPSE    #
############### 
#
# The Cosmology considered had the following paremeters:
# 
# - Matter Power Spectrum input file: "../test/datatest/file_pk.txt"
# - F window function input file: "../test/datatest/F_REFERENCE.txt"
# - Background data input file: "../test/datatest/WideA_ZA_background.dat"
#
# - Basic CosmoParams considered: 
#	 z_min = 0.05 	 z_max = 0.2
#	 theta_max = 1.5707963267948966 [rad] 	 h_0 = 0.7
#	 Ω_b = 0.0489 	 Ω_cdm = 0.25102 	 Ω_M0 = 0.29992
#
# - CosmoParams about the Input Power Spectrum: 
#	 fit_left_min = 1.0e-6 	 fit_right_min = 10.0 	 
#	 fit_left_max = 3.0e-6 	 fit_right_max = 20.0 	 
#
# - CosmoParams about the Input Power Spectrum Tools: 
#	 k_max = 10.0 	 N = 1024 	 fit_max = 0.5 	 
#	 con = true 	 k_min = 1.0e-8 	 fit_min = 0.05 	 
#
# - Computed quantities: 
# 	 effective redshift z_eff = 0.15045636096023224 
# 	 comoving s_min = 148.1920001465757 Mpc/h_0
# 	 comoving s_max = 571.7022420258767 Mpc/h_0
# 	 comoving s_eff = 435.

In [5]:
IMPLEMENTED_GR_EFFECTS = [
     "auto_doppler", "auto_lensing",
     "auto_localgp", "auto_integratedgp",

     "lensing_doppler", "doppler_lensing",
     "doppler_localgp", "localgp_doppler",
     "doppler_integratedgp",  "integratedgp_doppler",
     "lensing_localgp", "localgp_lensing",
     "lensing_integratedgp",  "integratedgp_lensing",
     "localgp_integratedgp", "integratedgp_localgp",
];

ξs_names = [
    L"$\xi^{v_{\parallel} \, v_{\parallel}}$", 
    L"$\xi^{\kappa \, \kappa}$", 
    L"$\xi^{\phi \, \phi}$", 
    L"$\xi^{\int\phi \, \int\phi}$", 
    
    L"$\xi^{\kappa \, v_{\parallel}}$",
    L"$\xi^{v_{\parallel} \, \kappa}$",
    
    L"$\xi^{v_{\parallel} \, \phi}$",
    L"$\xi^{\phi \, v_{\parallel}}$",
    
    L"$\xi^{v_{\parallel} \, \int\phi}$",
    L"$\xi^{\int\phi \, v_{\parallel}}$",
    
    L"$\xi^{\kappa \, \phi}$",
    L"$\xi^{\phi \, \kappa}$",
    
    L"$\xi^{\kappa \, \int\phi}$",
    L"$\xi^{\int\phi \, \kappa}$",
    
    L"$\xi^{\phi \, \int\phi}$",
    L"$\xi^{\int\phi \, \phi}$",
]

INDEX_GR_EFFECT = Dict([name => i for (i, name) in 
            enumerate(IMPLEMENTED_GR_EFFECTS)]...);
GR_EFFECT_INDEXED = Dict([i => name for (i, name) in 
            enumerate(IMPLEMENTED_GR_EFFECTS)]...);

In [6]:
effect = "lensing_doppler";
LIM_MULTIPOLE = 4;
l = @layout [A{0.05h}; a ; b c ; d e ];

In [7]:
my_path = "ALL_Multipoles_files/"
my_files = my_path .* [
    "xi_"* effect * "_L"* "$L" * ".txt" for L in 0:LIM_MULTIPOLE];

my_dict =  Dict([k => v for (k, v) in 
            zip(IMPLEMENTED_GR_EFFECTS, my_files)]...);

In [8]:
kwargs = Dict(
    :enhancer => 1e25, :N_μs => 30, :μ_atol => 0.0, :μ_rtol => 1e-2,
    :en => 1e6, :N_χs => 50, #Δχ_min = 1e-4, 
    );

for i in 0:(LIM_MULTIPOLE)
    GaPSE.print_map_ξ_multipole(
        cosmo, 
        "ALL_Multipoles_files/xi_"*effect*"_L$i" * ".txt", 
        effect, 
        10 .^ range(0, 3, length=300); 
        use_windows = false, L = i,
        kwargs...
    )
end

lensing_doppler, L=0: 100%|█████████████████████████████| Time: 0:00:03
lensing_doppler, L=1: 100%|█████████████████████████████| Time: 0:00:01
lensing_doppler, L=2: 100%|█████████████████████████████| Time: 0:00:01
lensing_doppler, L=3: 100%|█████████████████████████████| Time: 0:00:02
lensing_doppler, L=4: 100%|█████████████████████████████| Time: 0:00:02



time needed for map_integral_on_mu for lensing_doppler [in s] = 3.19687 

time needed for map_integral_on_mu for lensing_doppler [in s] = 1.47069 

time needed for map_integral_on_mu for lensing_doppler [in s] = 1.49246 

time needed for map_integral_on_mu for lensing_doppler [in s] = 2.13403 

time needed for map_integral_on_mu for lensing_doppler [in s] = 2.58924 


In [9]:
my_all_table = [readdlm(file, comments=true) for file in my_files]
my_all_ss = [convert(Vector{Float64}, t[:,1]) for t in my_all_table];
my_all_ξs = [convert(Vector{Float64}, t[:,2]) for t in my_all_table];

In [ ]:
pyplot()
ps = [plot() for i in 0:LIM_MULTIPOLE]

title = plot(title = effect * " multipoles", grid=false, 
    showaxis = false, bottom_margin = -20Plots.px, ticks=false)

legend_pos_norm=[:bottomleft, :topright, :bottomleft, :bottomleft, :topleft,]

legend_pos_square=[ :bottom, :topleft, :bottomleft, :bottomleft, :bottomright, ]

for i in 1:(LIM_MULTIPOLE+1)
    my_xs = my_all_ss[i]
    my_ys = my_all_ξs[i] .* my_xs .^2
    
    L=i-1
    ps[i] = plot(
        my_xs, my_ys .* my_xs .^2,
        size=(300, 100),
        lw=2, lc="red", ls=:solid,
        #label = "My",
        #legend=legend_pos_square[i],
        legend=false,
        xlabel = L"s \quad [h_0^{-1}\,\mathrm{Mpc}]",
        ylabel = L"s^2 \, " * ξs_names[INDEX_GR_EFFECT[effect]] * L"_{L=%$L}(s)",
        #xaxis=:log,
        #yaxis=:log,
        top_margins=15Plots.px,
    )
end

P = plot(title, ps..., layout=l, 
    left_margins = 15Plots.px, right_margins = 30Plots.px,
    size=(600,700))

#savefig(P, "xi_"*"$effect"*"_multipoles.pdf")
#display(P)